# ETL
 - O ETL será aplicado sob um banco de dados disponível em um repositório aberto

In [ ]:
# https://www.linkedin.com/pulse/etl-em-python-eou-r-hugo-souza/?originalSubdomain=pt

In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install quinn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import findspark
#inicializacao do Spark
findspark.init()
#Bibliotecas pyspark
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
import quinn
#Biblioteca para conexao com o PostgreSQL
import psycopg2

In [5]:
#Conexao com o Spark
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
sqlCtx = SQLContext(spark)

C:\spark\spark-3.5.0-bin-hadoop3\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Extração
 - A fase de Extração é realizada com a função SQLContext para ler o arquivo obtido em formato ".dat"

In [33]:
#url="https://archive.ics.uci.edu/dataset/43/haberman+s+survival/haberman.data"
url="C:/Users/User/Desktop/Portfolio_Geilsa/ETL/haberman.data"


data = sqlCtx.read.options(header='False').csv(url)
print(data)
data.show(10)

C:\spark\spark-3.5.0-bin-hadoop3\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]
+---+---+---+---+
|_c0|_c1|_c2|_c3|
+---+---+---+---+
| 30| 64|  1|  1|
| 30| 62|  3|  1|
| 30| 65|  0|  1|
| 31| 59|  2|  1|
| 31| 65|  4|  1|
| 33| 58| 10|  1|
| 33| 60|  0|  1|
| 34| 59|  0|  2|
| 34| 66|  9|  2|
| 34| 58| 30|  1|
+---+---+---+---+
only showing top 10 rows



#  Transformação 
    - Adicionando o nome das colunas

In [34]:
dataM = data.withColumnRenamed('_c0','IdadeOperado')\
            .withColumnRenamed('_c1','AnoOperacao')\
            .withColumnRenamed('_c2','QntNodulos')\
            .withColumnRenamed('_c3','Status')

dataM.show(10)

+------------+-----------+----------+------+
|IdadeOperado|AnoOperacao|QntNodulos|Status|
+------------+-----------+----------+------+
|          30|         64|         1|     1|
|          30|         62|         3|     1|
|          30|         65|         0|     1|
|          31|         59|         2|     1|
|          31|         65|         4|     1|
|          33|         58|        10|     1|
|          33|         60|         0|     1|
|          34|         59|         0|     2|
|          34|         66|         9|     2|
|          34|         58|        30|     1|
+------------+-----------+----------+------+
only showing top 10 rows



In [42]:
# Filtrar apenas dados com Starus = 1
dataSobrevida = dataM[dataM['Status']=='1']
dataSobrevida.show()

+------------+-----------+----------+------+
|IdadeOperado|AnoOperacao|QntNodulos|Status|
+------------+-----------+----------+------+
|          30|         64|         1|     1|
|          30|         62|         3|     1|
|          30|         65|         0|     1|
|          31|         59|         2|     1|
|          31|         65|         4|     1|
|          33|         58|        10|     1|
|          33|         60|         0|     1|
|          34|         58|        30|     1|
|          34|         60|         1|     1|
|          34|         61|        10|     1|
|          34|         67|         7|     1|
|          34|         60|         0|     1|
|          35|         64|        13|     1|
|          35|         63|         0|     1|
|          36|         60|         1|     1|
|          36|         69|         0|     1|
|          37|         60|         0|     1|
|          37|         63|         0|     1|
|          37|         58|         0|     1|
|         

# Load
    - A próxima etapa é a carga, ou Load em inglês, para um banco de dados, aqui foi utilizado o PostgreSQL

In [6]:
import psycopg2

In [54]:
conn = psycopg2.connect(host="localhost", database="postgres",
                        user = "postgres",
                        password = "postBarata")
#abertura da conexao
cur = conn.cursor()

In [ ]:
cur.execute("""CREATE TABLE sobrevida
           ( IdadeOperado integer,
             AnoOperacao integer,
             QntNodulos integer,
             Status integer);""")

In [55]:
dataSobrevida.show()

+------------+-----------+----------+------+
|IdadeOperado|AnoOperacao|QntNodulos|Status|
+------------+-----------+----------+------+
|          30|         64|         1|     1|
|          30|         62|         3|     1|
|          30|         65|         0|     1|
|          31|         59|         2|     1|
|          31|         65|         4|     1|
|          33|         58|        10|     1|
|          33|         60|         0|     1|
|          34|         58|        30|     1|
|          34|         60|         1|     1|
|          34|         61|        10|     1|
|          34|         67|         7|     1|
|          34|         60|         0|     1|
|          35|         64|        13|     1|
|          35|         63|         0|     1|
|          36|         60|         1|     1|
|          36|         69|         0|     1|
|          37|         60|         0|     1|
|          37|         63|         0|     1|
|          37|         58|         0|     1|
|         

In [56]:
a = [tuple(x) for x in dataSobrevida.collect()]
b = ','.join(['%s'] * len(a))
q = "INSERT INTO sobrevida (IdadeOperado,AnoOperacao,QntNodulos,Status) VALUES {}".format(b)
cur.execute(q, a)

In [58]:
#criacao de uma consulta a dados inseridos
cur.execute("SELECT * FROM sobrevida;")
result = cur.fetchmany(5)
for row in result:
    print(row)
conn.commit()

(30, 64, 1, 1)
(30, 62, 3, 1)
(30, 65, 0, 1)
(31, 59, 2, 1)
(31, 65, 4, 1)


In [59]:
#fechar cursor e conexao
cur.close()
conn.close()

# Extraindo dados a partir de requisições
    - PAG. 81

In [7]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [15]:
import http.client
import pandas as pd

In [16]:
# URL dos Dados: NYC Open Data website’s public HTTP APIs
url = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500"

In [17]:
import requests

# Defina a URL desejada
url = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500"

# Faça a requisição GET para a URL especificada
response = requests.get(url)

# Verifique se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    # Se a requisição foi bem-sucedida, obtenha os dados da resposta no formato JSON
    data = response.json()
    # Agora 'data' conterá os dados da resposta no formato JSON
else:
    # Se a requisição falhou, imprima uma mensagem de erro
    print("Erro ao fazer a requisição:", response.status_code)


In [18]:
from pyspark.sql.types import StructType, StructField, StringType

In [19]:
# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("Exemplo de conversão para DataFrame Spark") \
    .getOrCreate()

# Defina o esquema manualmente com base na estrutura dos dados
schema = StructType([
    StructField("crash_date", StringType(), True),
    StructField("crash_time", StringType(), True),
    StructField("on_street_name", StringType(), True),
    StructField("off_street_name", StringType(), True),
    StructField("number_of_persons_injured", StringType(), True),
    StructField("number_of_persons_killed", StringType(), True),
    StructField("number_of_pedestrians_injured", StringType(), True),
    StructField("number_of_pedestrians_killed", StringType(), True),
    StructField("number_of_cyclist_injured", StringType(), True),
    StructField("number_of_cyclist_killed", StringType(), True),
    StructField("number_of_motorist_injured", StringType(), True),
    StructField("number_of_motorist_killed", StringType(), True),
    StructField("contributing_factor_vehicle_1", StringType(), True),
    StructField("contributing_factor_vehicle_2", StringType(), True),
    StructField("collision_id", StringType(), True),
    StructField("vehicle_type_code1", StringType(), True),
    StructField("vehicle_type_code2", StringType(), True)
])

# Converter o DataFrame Pandas para um DataFrame Spark com o esquema fornecido
df_spark = spark.createDataFrame(data, schema)

# Visualizar os dados do DataFrame Spark
df_spark.show()


+--------------------+----------+--------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+
|          crash_date|crash_time|      on_street_name|     off_street_name|number_of_persons_injured|number_of_persons_killed|number_of_pedestrians_injured|number_of_pedestrians_killed|number_of_cyclist_injured|number_of_cyclist_killed|number_of_motorist_injured|number_of_motorist_killed|contributing_factor_vehicle_1|contributing_factor_vehicle_2|collision_id|  vehicle_type_code1|  vehicle_type_code2|
+--------------------+----------+--------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-----

# Selecionar algumas variáveis

In [25]:
df_spark = df_spark.select('crash_date', 'on_street_name',"off_street_name","number_of_persons_injured")
df_spark.show()

+--------------------+--------------------+--------------------+-------------------------+
|          crash_date|      on_street_name|     off_street_name|number_of_persons_injured|
+--------------------+--------------------+--------------------+-------------------------+
|2021-09-11T00:00:...|WHITESTONE EXPRES...|           20 AVENUE|                        2|
|2022-03-26T00:00:...|QUEENSBORO BRIDGE...|                NULL|                        1|
|2022-06-29T00:00:...|  THROGS NECK BRIDGE|                NULL|                        0|
|2021-09-11T00:00:...|                NULL|                NULL|                        0|
|2021-12-14T00:00:...|     SARATOGA AVENUE|      DECATUR STREET|                        0|
|2021-04-14T00:00:...|MAJOR DEEGAN EXPR...|                NULL|                        0|
|2021-12-14T00:00:...|BROOKLYN QUEENS E...|                NULL|                        0|
|2021-12-14T00:00:...|                NULL|                NULL|                        2|

In [26]:
df_spark = df_spark.fillna('NULO')
df_spark.show()

+--------------------+--------------------+--------------------+-------------------------+
|          crash_date|      on_street_name|     off_street_name|number_of_persons_injured|
+--------------------+--------------------+--------------------+-------------------------+
|2021-09-11T00:00:...|WHITESTONE EXPRES...|           20 AVENUE|                        2|
|2022-03-26T00:00:...|QUEENSBORO BRIDGE...|                NULO|                        1|
|2022-06-29T00:00:...|  THROGS NECK BRIDGE|                NULO|                        0|
|2021-09-11T00:00:...|                NULO|                NULO|                        0|
|2021-12-14T00:00:...|     SARATOGA AVENUE|      DECATUR STREET|                        0|
|2021-04-14T00:00:...|MAJOR DEEGAN EXPR...|                NULO|                        0|
|2021-12-14T00:00:...|BROOKLYN QUEENS E...|                NULO|                        0|
|2021-12-14T00:00:...|                NULO|                NULO|                        2|

In [27]:
conn = psycopg2.connect(host="localhost", database="postgres",
                        user = "postgres",
                        password = "postBarata")
#abertura da conexao
cur = conn.cursor()

In [71]:
#cur.execute("ROLLBACK;")

In [ ]:
cur.execute("""CREATE TABLE nyc
           ( crash_date VARCHAR,
             on_street_name VARCHAR,
             off_street_name VARCHAR,
             number_of_persons_injured VARCHAR);""")

In [28]:
a = [tuple(x) for x in df_spark.collect()]
b = ','.join(['%s'] * len(a))
q = "INSERT INTO nyc (crash_date,on_street_name,off_street_name,number_of_persons_injured) VALUES {}".format(b)
cur.execute(q, a)

In [29]:
#fechar cursor e conexao
cur.close()
conn.close()

# Puxando tabela do PostgreSQL

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Importar a biblioteca psycopg2
import psycopg2

# Crie uma sessão do Spark
spark = SparkSession.builder \
    .appName("Lendo dados do PostgreSQL e convertendo para DataFrame Spark") \
    .getOrCreate()

# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postBarata",
    host="localhost"
)

# Criar um cursor
cur = conn.cursor()

# Consulta para selecionar os dados da tabela
query = "SELECT * FROM nyc LIMIT 10;"  # Limitando a 10 linhas apenas para exemplo

# Executar a consulta
cur.execute(query)

# Recuperar os nomes das colunas
columns = [desc[0] for desc in cur.description]

# Recuperar os resultados
rows = cur.fetchall()

# Fechar o cursor e a conexão
cur.close()
conn.close()

# Criar um schema para o DataFrame Spark
schema = StructType([StructField(col_name, StringType(), True) for col_name in columns])

# Criar um DataFrame Spark a partir dos resultados da consulta
df = spark.createDataFrame(rows, schema)

# Exibir os dados do DataFrame Spark
df.show()


+--------------------+--------------------+---------------+-------------------------+
|          crash_date|      on_street_name|off_street_name|number_of_persons_injured|
+--------------------+--------------------+---------------+-------------------------+
|2021-09-11T00:00:...|WHITESTONE EXPRES...|      20 AVENUE|                        2|
|2022-03-26T00:00:...|QUEENSBORO BRIDGE...|            NaN|                        1|
|2022-06-29T00:00:...|  THROGS NECK BRIDGE|            NaN|                        0|
|2021-09-11T00:00:...|                 NaN|            NaN|                        0|
|2021-12-14T00:00:...|     SARATOGA AVENUE| DECATUR STREET|                        0|
|2021-04-14T00:00:...|MAJOR DEEGAN EXPR...|            NaN|                        0|
|2021-12-14T00:00:...|BROOKLYN QUEENS E...|            NaN|                        0|
|2021-12-14T00:00:...|                 NaN|            NaN|                        2|
|2021-12-14T00:00:...|                 NaN|           